In [47]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TweetTokenizer
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import geograpy as gg
import pycountry as pc

In [17]:
df = pd.read_csv("output.csv")
df.head()

,user,location,tweet,timeStamp
0,@BridportMan,Bridport,We'll be on Christmas break from December 16th...,5:54 AM - 27 Nov 2019
1,@HILeicesterCity,"Leicester, England",Plan your pre-Christmas break to Leicester now...,8:08 AM - 27 Nov 2019
2,@afaranwide,NaN,"Sunset over Phu Quoc, Vietnam.\n\nLook out for...",2:41 AM - 27 Nov 2019
3,@Sarah___Mary,South Wales,When you realise you only have to survive one ...,11:21 PM - 26 Nov 2019
4,@EduClickAfrica,"Centre, Cameroon",Our #Techlab offers affordable #excursion expe...,12:05 AM - 25 Nov 2019


In [33]:
tknzr = TweetTokenizer()
df['location_filled'] = df['location'].fillna("")
df['Location_Token'] = df['location_filled'].apply(lambda x: word_tokenize(x))
df['Tweet_Token'] = df['tweet'].apply(lambda x: tknzr.tokenize(x))

In [ ]:
def find_country(list):
    for country in pycountry.countries:
        if country.name in list:
            print(country.name)

In [36]:
countries_list = []
for country in list(pc.countries):
    countries_list.append(country.alpha_2.lower())
    countries_list.append(country.alpha_3.lower())
    countries_list.append(country.name.lower())
    try:
        countries_list.append(country.common_name.lower())
    except:
        None
city_list = []
for city in list(pc.subdivisions):
    city_list.append(city.name.lower())
print(countries_list[0:10])
print(city_list[0:10])

['aw', 'abw', 'aruba', 'af', 'afg', 'afghanistan', 'ao', 'ago', 'angola', 'ai']
['canillo', 'encamp', 'la massana', 'ordino', 'sant julià de lòria', 'andorra la vella', 'escaldes-engordany', "'ajmān", 'abū ȥaby [abu dhabi]', 'dubayy']


In [38]:
def country_name(list):
    if list == "":
        return ""
    else:
        for value in list:
            value = value.lower()
            fuzz_score_country = {}
            for country in countries_list:
                fuzz_score_country[country] = fuzz.ratio(value,country)
            for country in ['united states of america', 'murica', 'u.s.a.', 'usausausa','america', 'us of a','usa ',' usa', 'u s','u s a','u.s.','trumpistan','usa usa usa','pittsburgh','unhinged states','murrika','n. america','ahem....amerca','usa usa usa!!!!']:
                fuzz_score_country[country] = fuzz.ratio(value,country)
            for country in ['the netherlands','england','u.k.','uk','uk ',' uk','uae','korea','south korea']:
                fuzz_score_country[country] = fuzz.ratio(value,country)
            if max(fuzz_score_country, key=fuzz_score_country.get) in ['cascadia','united states of america', 'murica', 'u.s.a.', 'usausausa','america', 'us of a','usa ',' usa', 'u s','u s a','u.s.','trumpistan','usa usa usa','pittsburgh','unhinged states','murrika','n. america','ahem....amerca','usa usa usa!!!!']:
                return pc.countries.lookup('united states').alpha_3
            elif max(fuzz_score_country, key=fuzz_score_country.get) in ['england','u.k.','uk','uk ',' uk']:
                return pc.countries.lookup('united kingdom').alpha_3
            elif max(fuzz_score_country, key=fuzz_score_country.get) in ['uae']:
                return pc.countries.lookup('United Arab Emirates').alpha_3
            elif max(fuzz_score_country, key=fuzz_score_country.get) in ['korea','south korea']:
                return pc.countries.lookup('KR').alpha_3
            elif max(fuzz_score_country, key=fuzz_score_country.get) in ['the netherlands']:
                return pc.countries.lookup('netherlands').alpha_3
            elif fuzz_score_country[max(fuzz_score_country, key=fuzz_score_country.get)] <86:
                fuzz_score_city = {}
                for city in city_list:
                    fuzz_score_city[city] = fuzz.ratio(value,city)
                if fuzz_score_city[max(fuzz_score_city, key=fuzz_score_city.get)] <90:
                    return np.nan
                else:
                    return pc.countries.lookup(pc.subdivisions.lookup(max(fuzz_score_city, key=fuzz_score_city.get)).country_code).alpha_3
            return pc.countries.lookup(max(fuzz_score_country, key=fuzz_score_country.get)).alpha_3


In [ ]:
for 
df['Location_Token'][0]

In [54]:
geograpy.get_place_context(text = df['Location_Token'][0][0])

LookupError: 
**********************************************************************
  Resource [93mmaxent_ne_chunker[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('maxent_ne_chunker')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mchunkers/maxent_ne_chunker/PY3/english_ace_multiclass.pickle[0m

  Searched in:
    - '/Users/jass/nltk_data'
    - '/Users/jass/anaconda3/nltk_data'
    - '/Users/jass/anaconda3/share/nltk_data'
    - '/Users/jass/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [44]:
for city in city_list:
    print(city) if city == 'Bridport' else None

In [40]:
# df['Location_Classify'] = df_process['location'].map(lambda value: country_name(value))
country_name(df['Location_Token'][0])

nan

In [12]:
import nltk
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import pycountry as pc
countries_list = []
for country in list(pc.countries):
    countries_list.append(country.alpha_2.lower())
    countries_list.append(country.alpha_3.lower())
    countries_list.append(country.name.lower())
    try:
        countries_list.append(country.common_name.lower())
    except:
        None
city_list = []
for city in list(pc.subdivisions):
    city_list.append(city.name.lower())
    
def country_name(value):
    if value is np.nan:
        return np.nan
    elif isinstance(value,int):
        return np.nan
    else:
        value = value.lower()
        fuzz_score_country = {}
        for country in countries_list:
            fuzz_score_country[country] = fuzz.ratio(value,country)
        for country in ['united states of america', 'murica', 'u.s.a.', 'usausausa','america', 'us of a','usa ',' usa', 'u s','u s a','u.s.','trumpistan','usa usa usa','pittsburgh','unhinged states','murrika','n. america','ahem....amerca','usa usa usa!!!!']:
            fuzz_score_country[country] = fuzz.ratio(value,country)
        for country in ['the netherlands','england','u.k.','uk','uk ',' uk','uae','korea','south korea']:
            fuzz_score_country[country] = fuzz.ratio(value,country)
        if max(fuzz_score_country, key=fuzz_score_country.get) in ['cascadia','united states of america', 'murica', 'u.s.a.', 'usausausa','america', 'us of a','usa ',' usa', 'u s','u s a','u.s.','trumpistan','usa usa usa','pittsburgh','unhinged states','murrika','n. america','ahem....amerca','usa usa usa!!!!']:
            return pc.countries.lookup('united states').alpha_3
        elif max(fuzz_score_country, key=fuzz_score_country.get) in ['england','u.k.','uk','uk ',' uk']:
            return pc.countries.lookup('united kingdom').alpha_3
        elif max(fuzz_score_country, key=fuzz_score_country.get) in ['uae']:
            return pc.countries.lookup('United Arab Emirates').alpha_3
        elif max(fuzz_score_country, key=fuzz_score_country.get) in ['korea','south korea']:
            return pc.countries.lookup('KR').alpha_3
        elif max(fuzz_score_country, key=fuzz_score_country.get) in ['the netherlands']:
            return pc.countries.lookup('netherlands').alpha_3
        elif fuzz_score_country[max(fuzz_score_country, key=fuzz_score_country.get)] <86:
            fuzz_score_city = {}
            for city in city_list:
                fuzz_score_city[city] = fuzz.ratio(value,city)
            if fuzz_score_city[max(fuzz_score_city, key=fuzz_score_city.get)] <90:
                return np.nan
            else:
                return pc.countries.lookup(pc.subdivisions.lookup(max(fuzz_score_city, key=fuzz_score_city.get)).country_code).alpha_3
        return pc.countries.lookup(max(fuzz_score_country, key=fuzz_score_country.get)).alpha_3

# df_process['country'] = df_process['country'].fillna('')
df_process['location'] = df_process['location'].map(lambda value: country_name(value))

In [13]:
df_process['location']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      GBR
9      NaN
10     NaN
11     USA
12     NaN
13     NaN
14     GBR
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     GBR
22     USA
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
      ... 
562    NaN
563    NaN
564    NaN
565    NaN
566    NaN
567    NaN
568    USA
569    NaN
570    NaN
571    NaN
572    NaN
573    NaN
574    NaN
575    NaN
576    NaN
577    NaN
578    NaN
579    NaN
580    NaN
581    NaN
582    NaN
583    USA
584    NaN
585    NaN
586    NaN
587    NaN
588    NaN
589    NaN
590    NaN
591    NaN
Name: location, Length: 592, dtype: object